# Data Prep and Wrangling
- It's important to every data analytics and science pipeline
- It's the process of cleaning, structuring, and enriching raw data into desired format for better EDA, statistical analysis, and machine learning.
- Components:
    - Data acquisition: Gathering data from different sources
        - Internal data sources: grabbing local company files, connecting to the company's database/data warehouse (proprietary to the company)
        - External data sources: grabbing data files from the web, connecting to a public server, or subscription to a data vendor (proprietary):
            - https://data.gov
            - https://kaggle.com
            - https://archive.ics.uci.edu/

    - Data Preparation:
        - Selecting and combining data:
            - Slicing nd filtering data
            - Integrating data: joins or unions 
        - Data Cleansing: 
            - Dropping unnecessary columns
            - Changing or formatting data types (e.g. string to date)
            - Treating missing values
            - Treating outliers
            - Removing duplicates
            - Inconsistent values in the data
        - Data transformation and feature engineering
            - Data aggregation e.g. daily transactions to weekly
            - Feature extraction: e.g. extract age bands from age (continuous to categorical), deriving a column value based on existing columns. e.g. create a new indicator from an existing "compound" column that indicates whether Carbon exists or not. 
            - Label encoding
            - Data scaling (standardization)
- Importance of Data Wrangling and Preprocessing for Machine Learning:
    - Improves data quality and ensuring the data is reliable, accurate, and complete
    - Helps with data interpretation
    - Reducing bias and errors
    - Improves machine learning model performance


## Exercise

### Data Dictionary
**Description of Attributes:**

* **instant** - event or instant id
* **dteday** - date of the rental/ride 
* **season** -  
    - 1 = spring
    - 2 = summer
    - 3 = fall
    - 4 = winter 
* **yr** 
* **mnth**
* **hr** hour of the rental
* **holiday** - whether the day is considered a holiday
* **weekday** -
* **weathersit** 
    * 1: Clear, Few clouds, Partly cloudy, Partly cloudy
    * 2: Mist + Cloudy, Mist + Broken clouds, Mist + Few clouds, Mist
    * 3: Light Snow, Light Rain + Thunderstorm + Scattered clouds, Light Rain + Scattered clouds
    * 4: Heavy Rain + Ice Pallets + Thunderstorm + Mist, Snow + Fog 
* **temp** - temperature in Celsius
* **atemp** - "feels like" temperature in Celsius
* **hum** - relative humidity
* **windspeed** - wind speed
* **casual** - number of non-registered user rentals initiated
* **registered** - number of registered user rentals initiated
* **cnt** - number of total rentals (casual and registered)


In [38]:
data_path = '/Users/bassel_instructor/Documents/Datasets/'

In [39]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [40]:
dataset_1 = pd.read_csv(data_path+'rental_bike_descr.csv')
dataset_1.head()

,instant,dteday,season,yr,mnth,hr,holiday,weekday,weathersit,temp
0,1,01-01-2011,1,0,1,0,False,6,1,0.24
1,2,01-01-2011,1,0,1,1,False,6,1,0.22
2,3,01-01-2011,1,0,1,2,False,6,1,0.22
3,4,01-01-2011,1,0,1,3,False,6,1,0.24
4,5,01-01-2011,1,0,1,4,False,6,1,0.24


- All rentals are from the same day, but looks like we have multiple transactions per day
- Need to investigate the season field because it says 1 for Spring, but the data is in Jan. Observe the temperature to double check (its 25 for ex)
- New Year's Day should be a holiday. Can be fixed using data wrangling 
- Temperature is in decimal (for Celsius)
- Missing columns based on the data dictionary (e.g. atemp, hum, cnt)


Since we're missing some columns, we need to look for them in a different dataset. Therefore, we need to potentially join our datasets.

- For the next data set we need to use `read_excel`. Some machines require `openpyxl` to be installed.
- Run `pip install openpyxl`

In [41]:
dataset_2 = pd.read_excel(data_path+'rental_bike_season.xlsx')
dataset_2.head()

,Unnamed: 0,instant,atemp,hum,windspeed,casual,registered,cnt
0,0,1,0.2879,0.81,0.0,3,13,16
1,1,2,0.2727,0.80,0.0,8,32,40
2,2,3,0.2727,0.80,0.0,5,27,32
3,3,4,0.2879,0.75,0.0,3,10,13
4,4,5,0.2879,0.75,0.0,0,1,1


- The dataset above completes the first one (dataset_1) we need to eventually join dataset_1 and dataset_2 to get the full dataset.
- 'Unnamed: 0' columns seems to be redundant. Therefore, we can drop it.

In [42]:
dataset_2 = dataset_2.drop(columns=['Unnamed: 0'], axis=1)
dataset_2.head()

,instant,atemp,hum,windspeed,casual,registered,cnt
0,1,0.2879,0.81,0.0,3,13,16
1,2,0.2727,0.80,0.0,8,32,40
2,3,0.2727,0.80,0.0,5,27,32
3,4,0.2879,0.75,0.0,3,10,13
4,5,0.2879,0.75,0.0,0,1,1


## Slicing and Filtering The Data

In [43]:
dataset_2.loc[100:200, ['instant', 'hum']]

,instant,hum
100,101,0.37
101,102,0.37
102,103,0.33
103,104,0.33
104,105,0.30
...,...,...
196,197,0.40
197,198,0.37
198,199,0.34
199,200,0.32


In [44]:
# select the first 300 rows and first 4 columns in the data
dataset_2.iloc[:300, :4]

,instant,atemp,hum,windspeed
0,1,0.2879,0.81,0.0000
1,2,0.2727,0.80,0.0000
2,3,0.2727,0.80,0.0000
3,4,0.2879,0.75,0.0000
4,5,0.2879,0.75,0.0000
...,...,...,...,...
295,296,0.1818,0.40,0.3284
296,297,0.1515,0.47,0.2537
297,298,0.1515,0.47,0.2239
298,299,0.1212,0.46,0.2985


In [45]:
# get the rows where casual is above 3
dataset_2[dataset_2['casual']>3]

,instant,atemp,hum,windspeed,casual,registered,cnt
1,2,0.2727,0.80,0.0000,8,32,40
2,3,0.2727,0.80,0.0000,5,27,32
9,10,0.3485,0.76,0.0000,8,6,14
10,11,0.3939,0.76,0.2537,12,24,36
11,12,0.3333,0.81,0.2836,26,30,56
...,...,...,...,...,...,...,...
580,581,0.1970,0.93,0.3284,6,35,41
581,582,0.1970,0.93,0.3284,7,41,48
582,583,0.1970,0.93,0.3284,4,43,47
591,592,0.2121,0.74,0.0896,4,55,59


In [46]:
dataset_2[dataset_2['casual']>3].min()

instant        2.0000
atemp          0.0606
hum            0.2100
windspeed      0.0000
casual         4.0000
registered     6.0000
cnt           13.0000
dtype: float64

**Logical Operators**
if a is the col and b is the value</br> 
![logOp](https://miro.medium.com/v2/resize:fit:640/1*H9m-yjLwZ5-M16qld4fvOA.png)

In [47]:
# Find the rentals that are 3 for casual and greater than 10 for registered users
my_filter = (dataset_2['casual']==3) & (dataset_2['registered']>10)

dataset_2[my_filter].sample(10)

,instant,atemp,hum,windspeed,casual,registered,cnt
261,262,0.1667,0.59,0.1642,3,125,128
547,548,0.1515,0.64,0.2537,3,21,24
590,591,0.2273,0.69,0.0896,3,76,79
240,241,0.1970,0.51,0.1642,3,127,130
380,381,0.1515,0.47,0.2239,3,30,33
535,536,0.1212,0.46,0.1343,3,43,46
239,240,0.2121,0.51,0.0896,3,214,217
399,400,0.2273,0.87,0.1642,3,32,35
488,489,0.0606,0.41,0.1642,3,50,53
295,296,0.1818,0.40,0.3284,3,141,144


In [48]:
# select casual values of 4,7, 10
my_filter = dataset_2['casual'].isin([4,7,10])
dataset_2[my_filter].sample(5)

,instant,atemp,hum,windspeed,casual,registered,cnt
313,314,0.1970,0.37,0.3881,4,73,77
565,566,0.3182,0.49,0.0896,7,218,225
106,107,0.2879,0.28,0.1940,7,55,62
581,582,0.1970,0.93,0.3284,7,41,48
349,350,0.2727,0.56,0.1343,4,19,23


In [49]:
# not equal to 7
my_filter = dataset_2['casual'].ne(7)
dataset_2[my_filter].sample(5)

,instant,atemp,hum,windspeed,casual,registered,cnt
589,590,0.2273,0.69,0.0896,3,101,104
287,288,0.1364,0.50,0.3582,2,36,38
112,113,0.1970,0.51,0.1940,3,40,43
597,598,0.2121,0.75,0.1642,0,2,2
43,44,NaN,0.39,0.3582,5,17,22


Get all the rows that are below average humidity

In [50]:
hum_mean = dataset_2['hum'].mean()
dataset_2[dataset_2['hum'] < hum_mean]

,instant,atemp,hum,windspeed,casual,registered,cnt
41,42,0.3333,0.46,0.3284,10,43,53
42,43,0.2879,0.42,0.4478,1,29,30
43,44,NaN,0.39,0.3582,5,17,22
44,45,0.2273,0.44,0.3284,11,20,31
45,46,0.2121,0.44,0.2985,0,9,9
...,...,...,...,...,...,...,...
563,564,0.3485,0.42,0.0000,14,102,116
564,565,0.3333,0.45,0.0000,14,208,222
565,566,0.3182,0.49,0.0896,7,218,225
587,588,0.2273,0.55,0.1940,1,23,24


### Data Integration
- Merging `merge()`: similar to joining tables in Excel ro SQL. Horizontal integration
- Concatenating `concat()`: similar to union in SQL (stacking tables vertically)

![joins](https://statisticsglobe.com/wp-content/uploads/2021/12/join-types-python-merge-programming.png)

![sql2](https://miro.medium.com/v2/resize:fit:1200/1*9eH1_7VbTZPZd9jBiGIyNA.png)

#### Data Merge (Horizontal Integration)
- dataset_1 and dataset_2 need to be joined merged so we can have all columns in one table
- to join them successfully:
    - check for missing data e.g. one table has more rows than the other (or different ID values)
    - both tables need to have a consistent common factor (ID/key)

In [51]:
len(dataset_1) == len(dataset_2)

True

Both have the same number of rows.

In [52]:
(dataset_1['instant'].sort_values() == dataset_2['instant'].sort_values() ).sum()

610

We got 610 True values, which means all rows have the same aligned instant values. 

In [53]:
# check for duplicates
dataset_1.duplicated().sum()

0

In [54]:
dataset_2.duplicated().sum()

0

Both datasets showed 0 duplicates

So in this case, you could join the datasets using any of the 4 types of joins

In [55]:
combined_dataset_1_2 = pd.merge(left=dataset_1, right=dataset_2,
                                how='inner', #any type works here since we have same num of instant values and they match
                                on='instant')
combined_dataset_1_2

,instant,dteday,season,yr,mnth,hr,holiday,weekday,weathersit,temp,atemp,hum,windspeed,casual,registered,cnt
0,1,01-01-2011,1,0,1,0,False,6,1,0.24,0.2879,0.81,0.0000,3,13,16
1,2,01-01-2011,1,0,1,1,False,6,1,0.22,0.2727,0.80,0.0000,8,32,40
2,3,01-01-2011,1,0,1,2,False,6,1,0.22,0.2727,0.80,0.0000,5,27,32
3,4,01-01-2011,1,0,1,3,False,6,1,0.24,0.2879,0.75,0.0000,3,10,13
4,5,01-01-2011,1,0,1,4,False,6,1,0.24,0.2879,0.75,0.0000,0,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
605,606,28-01-2011,1,0,1,11,False,5,3,0.18,0.2121,0.93,0.1045,0,30,30
606,607,28-01-2011,1,0,1,12,False,5,3,0.18,0.2121,0.93,0.1045,1,28,29
607,608,28-01-2011,1,0,1,13,False,5,3,0.18,0.2121,0.93,0.1045,0,31,31
608,609,28-01-2011,1,0,1,14,False,5,3,0.22,0.2727,0.80,0.0000,2,36,38


In [56]:
combined_dataset_1_2.duplicated().sum()

0

In [57]:
combined_dataset_1_2.shape

(610, 16)

#### Data Concatenation (Vertical Integration)

In [58]:
dataset_3 = pd.read_csv(data_path+'rental_bike_dataset_ext.csv')
dataset_3.head()

,instant,dteday,season,yr,mnth,hr,holiday,weekday,weathersit,temp,atemp,hum,windspeed,casual,registered,cnt
0,620,29-01-2011,1,0,1,1,False,6,1,0.22,0.2273,0.64,0.1940,0,20,20
1,621,29-01-2011,1,0,1,2,False,6,1,0.22,0.2273,0.64,0.1642,0,15,15
2,622,29-01-2011,1,0,1,3,False,6,1,0.20,0.2121,0.64,0.1343,3,5,8
3,623,29-01-2011,1,0,1,4,False,6,1,0.16,0.1818,0.69,0.1045,1,2,3
4,624,29-01-2011,1,0,1,6,False,6,1,0.16,0.1818,0.64,0.1343,0,2,2


**observation**
- we see a similar structure to the combined dataset
- the instant column appears to be a continuation of the instant column from the combined dataset
- notice that combined data ends at instant 610, but dataset_3 starts at instant 620. where are the 10 missing instants? 

In [59]:
combined_dataset_1_2['instant'].max()

610

In [60]:
dataset_3['instant'].min()

611

We can sort the data so we can have a continuous instant value when we concat.

In [61]:
dataset_3 = dataset_3.sort_values(by='instant').reset_index(drop=True)
dataset_3

,instant,dteday,season,yr,mnth,hr,holiday,weekday,weathersit,temp,atemp,hum,windspeed,casual,registered,cnt
0,611,28-01-2011,1,0,1,16,False,5,1,0.22,0.2727,0.80,0.0000,10,70,80
1,612,28-01-2011,1,0,1,17,False,5,1,0.24,0.2424,0.75,0.1343,2,147,149
2,613,28-01-2011,1,0,1,18,False,5,1,0.24,0.2273,0.75,0.1940,2,107,109
3,614,28-01-2011,1,0,1,19,False,5,2,0.24,0.2424,0.75,0.1343,5,84,89
4,615,28-01-2011,1,0,1,20,False,5,2,0.24,0.2273,0.70,0.1940,1,61,62
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
385,996,14-02-2011,1,0,2,3,False,1,1,0.34,0.3182,0.46,0.2239,1,1,2
386,997,14-02-2011,1,0,2,4,False,1,1,0.32,0.3030,0.53,0.2836,0,2,2
387,998,14-02-2011,1,0,2,5,False,1,1,0.32,0.3030,0.53,0.2836,0,3,3
388,999,14-02-2011,1,0,2,6,False,1,1,0.34,0.3030,0.46,0.2985,1,25,26


In [62]:
final_dataset = pd.concat([combined_dataset_1_2,dataset_3])
final_dataset.shape

(1000, 16)

> Note: Concat is similar to UNION in SQL:
```SQL
SELECT *
FROM dataset_1
UNION
SELECT *
FROM dataset_2
```

In [63]:
final_dataset.to_csv('finalOutput.csv', sep='|', index=False)

## DQA Techniques
- DQA: Data Quality Assurance are methods of checking the data to make sure it has no errors and ready for analysis/publishing
- In DS, it needs to be done before building the machine learning model and after importing the data and applying necessary changes. That said, it can also be done before data wrangling to figure out which parts need to be fixed/improved.
- Types validation:
    - check for nulls or missing values
    - check for duplicates
    - check for outliers (check if data ranges make sense)
    - check for inconsistent data
    - check for data completeness (Using references or subject mater expertise)
    - check for proper data types
    - string value consistency e.g. Street or ST. Or ST

Let's perform a sanity check for the finalized bike rental data

In [64]:
# check if the results of the concat add up
len(final_dataset) == len(combined_dataset_1_2) + len(dataset_3)

True

In [65]:
# check what values we have for season and if we have any strange values
final_dataset['season'].unique()

array([1])

Looks like we have data for 1 season, if the goal of the project to analyze all seasons, we need to contact the data provider to ask for 2,3, and 4

In [67]:
#do the same check for months 
final_dataset['mnth'].unique()

array([1, 2])

In [68]:
final_dataset['cnt'].sum() == final_dataset['casual'].sum() + final_dataset['registered'].sum()

True

In [69]:
# temperature ranges
final_dataset['temp'].describe()

count    1000.000000
mean        0.205900
std         0.078977
min         0.020000
25%         0.160000
50%         0.200000
75%         0.240000
max         0.460000
Name: temp, dtype: float64

In [ ]:
final_dataset.isna().sum().sum() #isnull()
#using 2 sum() functions give you a total of all nulls in the data

11

In [72]:
#using 1 sum() gives you total by column name
final_dataset.isna().sum()

instant        0
dteday         0
season         0
yr             0
mnth           0
hr             0
holiday        0
weekday        0
weathersit     0
temp           0
atemp         11
hum            0
windspeed      0
casual         0
registered     0
cnt            0
dtype: int64

**Observations**
- We have 11 nulls for "atemp" column
- It's very important to address nulls before building a machine learning model. It can be done using 2 main methods:
    - Dropping the rows with nulls
    - Imputation (replacing nulls with a sensible value)
- The dataset we're using could be a good candidate for dropping the nulls. First, we have to prove that the ratio of nulls is very low
- If the nulls represent 1-5% of the dataset, it's recommended to drop the rows instead of imputation

In [74]:
# nulls/total
print('Ratio of nulls:',round(final_dataset.isna().sum().sum()/len(final_dataset)*100,2),'%')

Ratio of nulls: 1.1 %


Conclusion: we can drop the rows with nulls

In [75]:
final_dataset = final_dataset.dropna()
final_dataset.shape

(989, 16)

Renaming columns to more readable names

In [76]:
final_dataset.columns

Index(['instant', 'dteday', 'season', 'yr', 'mnth', 'hr', 'holiday', 'weekday',
       'weathersit', 'temp', 'atemp', 'hum', 'windspeed', 'casual',
       'registered', 'cnt'],
      dtype='object')

In [77]:
final_dataset.rename(columns={
                    'dteday':'rental_date',
                    'yr':'year', 'mnth':'month', 'hr':'hour'
                        }, inplace=True)

final_dataset.columns

/var/folders/3x/ds4ljhhn611fn_glg6pp102h0000gq/T/ipykernel_19648/2314003051.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final_dataset.rename(columns={


Index(['instant', 'rental_date', 'season', 'year', 'month', 'hour', 'holiday',
       'weekday', 'weathersit', 'temp', 'atemp', 'hum', 'windspeed', 'casual',
       'registered', 'cnt'],
      dtype='object')

Capitalize the first letter of every column name

In [78]:
final_dataset.columns = final_dataset.columns.str.title()
final_dataset.columns

Index(['Instant', 'Rental_Date', 'Season', 'Year', 'Month', 'Hour', 'Holiday',
       'Weekday', 'Weathersit', 'Temp', 'Atemp', 'Hum', 'Windspeed', 'Casual',
       'Registered', 'Cnt'],
      dtype='object')

## More about `dropna()`

In [79]:
data = {'a':[5,4,3,np.nan],
        'b':[np.nan,2,4,np.nan],
        'c':[np.nan,6,2,10],
        'd':[np.nan,4,np.nan,3],}

df = pd.DataFrame(data)
df

,a,b,c,d
0,5.0,NaN,NaN,NaN
1,4.0,2.0,6.0,4.0
2,3.0,4.0,2.0,NaN
3,NaN,NaN,10.0,3.0


In [80]:
# simple dropna will drop every row with at least 1 null
df.dropna()

,a,b,c,d
1,4.0,2.0,6.0,4.0


In [81]:
# keep at least 3 non-nulls
df.dropna(thresh=3)

,a,b,c,d
1,4.0,2.0,6.0,4.0
2,3.0,4.0,2.0,NaN


In [82]:
df.dropna(thresh=2) # criteria is 2 non nulls

,a,b,c,d
1,4.0,2.0,6.0,4.0
2,3.0,4.0,2.0,NaN
3,NaN,NaN,10.0,3.0
